In [ ]:
import pandas as pd
import numpy as np
import sys
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from thefuzz import process

%load_ext autoreload
%autoreload 2

sys.path.append("..")
from utils import transtable
import utils as plot



## Load Raw data

In [ ]:
data = pd.read_csv('../raw_data/census.csv')
gd = data.replace('nan', np.nan)
# gd = data[['timestamp','have_ordered', 'have_playdate', 'satisfied_with_season','season_feedback','spoiler_free','satisfied_with_games','most_exciting_game','season_game_feedback','purchase_season_2','pay_for_12_games','pay_for_24_games','have_purchased_game','quick_hit_price','short_price','standard_price','long_price','highest_price','known_developer','game_tags','session_10min','session_30min','session_60min','session_hour_plus','most_excited_game']]
gd = gd.astype({'season_feedback':str})
for col in ['have_ordered', 'have_playdate']:
    gd[col] = gd[col] == 'Yes'

In [ ]:
s1_games = '''
    whitewater wipeout 
    casual birder
    crankins time travel adventure 
    boogie loops
    lost your marbles
    pick pack pup
    flipper lifter
    echoic memory
    omaze
    demon quest 85
    zipper 
    hyper meteor
    questy chess
    executive golf dx 
    saturday edition
    star sled 
    inventory hero
    spellcorked 
    snak 
    sasquatchers 
    forrest byrnes up in smoke
    battleship godios
    b360  
    ratcheteer
'''
extra_vocab = '''
    surfin surfing surf bird birding cranking crank crankin loop golfing business office starsled spell corked sasquatcher 360 snake
'''
s1_games_list = [g.strip() for g in s1_games.split('\n')]
s1_games_list = set(s1_games_list)
s1_games_list.remove('')
game_vocab = (s1_games + extra_vocab).split()


In [ ]:
pd.DataFrame({'game':list(s1_games_list), 'week':[7, 6, 2, 9, 12, 1, 8, 5, 3, 11, 4, 3, 6, 1, 10, 12, 9, 5, 2, 7, 8, 10, 11, 4]}).to_csv('s1_games_week.csv',index=False)

In [ ]:

def normalize_game(val, vocab=None):
    if val is None or pd.isna(val): return None
    res = []
    val = val.lower().strip().translate(transtable)
    words = val.split()
    for word in words:
        if vocab and word in vocab:
            res.append(word)
        elif not vocab:
            res.append(word)
    
    return ' '.join(res)

def export_list_to_fix(data, col, vocab, games):
    favs = data[data[col].notna()].copy()
    favs['normalized'] = favs[col].apply(lambda x: normalize_game(x, vocab))
    favs['inspect'] = favs.normalized.apply(lambda x: x not in games)
    favs[[col, 'normalized', 'inspect']].to_excel(col+'.xlsx')

    return favs

def import_fixed_list(filename, source_df):
    source_df = source_df.copy()
    fixed_favs = pd.read_csv(filename, index_col=0).normalized
    source_df['fixed'] = fixed_favs
    source_df = source_df[['timestamp', 'fixed','have_playdate','have_ordered']]
    # source_df = source_df.fillna('')
    return source_df

def get_freq_count(s, multiple_choice=True):
    s = s[s.notna()]
    def make_item_list():
        return [x.strip() for x in ' | '.join(s).split('|') if x.strip() != '' ]

    from collections import Counter
    fav_items = s.values
    if multiple_choice:
        fav_items = make_item_list()
    fav_items = Counter(fav_items).most_common()
    return fav_items




# Get Normalized List of S1 Games

In [ ]:
temp = export_list_to_fix(gd, 'most_exciting_game', game_vocab, s1_games_list)
fav_s1_games = import_fixed_list('fixed_s1_games.csv', temp)

# Identify common non S1 Games

In [ ]:
upcoming_game_comments = gd[gd.most_excited_game.notna()].most_excited_game

v = CountVectorizer(stop_words='english', strip_accents='ascii', ngram_range=(1,4))
bow = v.fit_transform(upcoming_game_comments)
sum_words = bow.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in v.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
top_words = words_freq[101:]
# vocab = list(zip(*top_words))[0]
upcoming_games = {
    'mars after midnight':'lucas pope',
    'daily driver':'gingerbeardman',
    'wastebraver tiny survival tale':'',
    'atlantic 41':'submarine',
    'playmaker':'',
    'grand tour legends':'',
    'p-racing':'p racing p-racer',
    'voyage':'',
    'giles goddard snowboarding':'chuhai labs',
    'pullfrog':'amano pull frog',
    'moonward':'',
    'botanist':'',
    'silver ball tactics':'silverball',
    'rocket bytes':'possibly axolotl possiblyaxolotl',
    'legend etad':'etud',
    'poolsuite fm':'',
    'kicooya':'',
    'skwish':'',
    'crank turismo':'',
    'faraway fairway':'',
    'tapeworm':'',
    'bloom':'',
    'mobware minigames':'wario ware',
    'space kitsune':'starfox chuhai labs vector 3d'

}
upcoming_vocab = " ".join(upcoming_games.keys()) + " " + " ".join(upcoming_games.values())
upcoming_vocab = upcoming_vocab.split()
print(list(upcoming_games.keys()))

## Pick closest game based on fuzzy search for each comment

In [ ]:
keys = [' '.join((k,v)).strip() for k,v in upcoming_games.items()]
values = upcoming_games.keys()
option_map = dict(zip(keys, values))

def pick_closest(val, choices):
    val = normalize_game(val)
    if val:
        closest = process.extractOne(val, choices)
        if closest[1] >= 60:
            return option_map[closest[0]]
        else:
            return 'other'
    else:
        return None
       
comments = gd[gd.most_excited_game.notna()].most_excited_game
gd['normalized_upcoming_game'] = comments.apply(lambda x: pick_closest(x, keys))

# Normalize other multiple choice columns 

In [ ]:
# temp = gd[['timestamp','have_playdate','have_ordered','most_excited_game','normalized_upcoming_game']].dropna()
# temp.to_excel('upcoming_games.xlsx')
upcoming_games = import_fixed_list('fixed_upcoming_games.csv', gd)
freqs = get_freq_count(upcoming_games.fixed, multiple_choice=False)
top_cats = list(zip(*freqs))[0]


In [ ]:
exciting_s1 = plot.explode_multiple_choice(fav_s1_games, 'fixed')
exciting_non_s1 = upcoming_games

In [ ]:
exciting_s1.to_csv('exciting_s1.csv',index=False)
exciting_non_s1.to_csv('exciting_non_s1.csv', index=False)

In [ ]:
other_indies = pd.read_csv('other_indie_consoles_fixed.csv', usecols=range(3))
def update_col_with_fixed(data, col, other_list = [], standard_vals=[]):
    data.loc[data.fixed.notna(), col] = data.fixed.apply(lambda x: x + '*' if type(x) is str and x not in standard_vals else x)
    data = data[data[col] != 'None*']
    data.loc[data[col].isin(other_list), col] = 'Other'
    return data
other_indies = update_col_with_fixed(other_indies, 'other_indie_consoles', other_list=('Modded Gameboy', 'Pocket Sprite', 'Analogue Super Nt'), standard_vals=[
    'Arduboy', 'Anbernic systems (RG552 ect)', 'Retroid Pocket', 'Analogue Pocket'
])
other_indies[['pdidx','other_indie_consoles']].to_csv('other_indie_consoles_cleaned.csv')
other_indies.other_indie_consoles.value_counts()

In [ ]:
def explode_col(df, col, standard_threshold=15):
    exploded = plot.explode_multiple_choice(gd, col, ',')
    standard_vals = exploded[col].value_counts().index[:standard_threshold].values
    exploded['inspect'] = ~exploded[col].isin(standard_vals)
    exploded = exploded[[col, 'inspect']]
    exploded.to_csv(f'{col}_exploded.csv', index_label='pdidx')
    return standard_vals

def read_fixed_and_clean(col, other_list=[], standard_vals=[], ncols=3):
    content = pd.read_csv(f'{col}_fixed.csv', usecols=range(ncols))
    content = update_col_with_fixed(content, col, other_list, standard_vals)
    content[['pdidx', col]].to_csv(f'{col}_cleaned.csv')
    return content

In [ ]:
explode_col(gd, 'playdate_content')
read_fixed_and_clean('playdate_content', standard_vals=[
    'Reddit', 'Twitter', 'Discord', 'Itch.io', 'Youtube', 'Official Playdate Forum', 'Playdate wiki', 'Twitch'
])


In [ ]:
standard_values = explode_col(gd, 'dev_tools', standard_threshold=14)
read_fixed_and_clean('dev_tools', standard_vals=standard_values, ncols=4)

In [ ]:
standard_values = explode_col(gd, 'dev_playdate_tools', standard_threshold=7)
standard_values[-1] = 'Roomy-Playdate'
read_fixed_and_clean('dev_playdate_tools', standard_vals=standard_values, ncols=4)